### 1 - Raw file pre-processing

- Filter rows corresponding to a transfer, based on whether an amount is present.
- Extract only useful columns.
- Organize upfront transfers. The data is split per year of support when we want the individual distinct money transfers.
- Organize annual & one-year commitment transfers.
- Filter out rows corresponding to future transfers.


In [ ]:
from pathlib import Path
import pandas as pd
from functools import reduce
import re
from datetime import date
import os
import sys
import django

# Add the parent directory to the system path and setup django
BASE_DIR = str(Path(os.getcwd()).resolve().parent.parent.parent.parent)

if BASE_DIR not in sys.path:
    sys.path.append(BASE_DIR)

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "backend_site.settings")

django.setup()

from tsosi.data.preparation.cleaning_utils import clean_number_value

FILE_INPUT_FOLDER = Path.home() / "Nextcloud/TSOSI_data/doab/0_raw/"
FILE_OUTPUT_FOLDER = Path.home() / "Nextcloud/TSOSI_data/doab/1_pre_processed/"
INPUT_NAME = "2025-02-11-DOAB_Library_Report.xlsx"
# INPUT_NAME = "2025-02-11-DOAB_Sponsorship_Report.xlsx"
OUTPUT_NAME = f"{INPUT_NAME.split(".")[0]}_pre_processed.xlsx"

AMOUNT_COLS_REGEX = [r"Annual amount \(([A-Z]{3})\)", r"Amount \(([A-Z]{3})\)"]
amount_columns: dict[str, str] = {}
currencies: list[str] = []
start_date_col = None
end_date_col = None


def get_final_cols(df: pd.DataFrame):
    compulsory_cols = [
        "Company",
        "Country",
        "date_start",
        "date_end",
        "Commitment period (years)",
        "Invoice preference",
    ]
    bonus_cols = ["Supporter type", "Agent", "Sponsorship level"]
    bonus_cols = [c for c in bonus_cols if c in df.columns]
    extra_cols = [f"amount_{currency}" for currency in amount_columns.keys()]
    return compulsory_cols + extra_cols + bonus_cols


def select_transfers(df: pd.DataFrame):
    """
    Filter out rows that do not correspond to a transfer +
    clean the amount fields
    """
    compulsory_columns = [
        "Company",
        "Country",
        *amount_columns.values(),
        "Invoice preference",
        "Commitment period (years)",
        start_date_col,
        end_date_col,
    ]

    bonus_columns = [
        "Year",
        "Supporter type",
        "Agent",
        "Support confirmation date",
        "Sponsorship level",
        "Sponsorship confirmation date",
    ]
    bonus_columns = [c for c in bonus_columns if c in df.columns]
    useful_columns = compulsory_columns + bonus_columns

    amount_mask = None
    for col in amount_columns.values():
        mask_part = ~df[col].isna()
        amount_mask = (
            mask_part if amount_mask is None else amount_mask | mask_part
        )

    date_mask = (~df[start_date_col].isna()) & (~df[end_date_col].isna())
    mask = amount_mask & date_mask
    df_filtered = df[mask][useful_columns].copy()

    print(
        f"Discarded {len(df) - len(df_filtered)} rows without amount "
        f"out of {len(df)} rows."
    )

    # Clean amount values (some are numbers, other are strings, ...)
    def clean_number_error(x):
        return clean_number_value(x, False, True)

    for col in amount_columns.values():
        df_filtered[col] = df_filtered[col].apply(clean_number_error)
    return df_filtered


def handle_one_year_transfers(transfers: pd.DataFrame):
    statuses = ["one year commitment", "annual, one year commitment"]
    one_year = transfers[
        transfers["Invoice preference"].str.lower().str.strip().isin(statuses)
    ].copy()

    one_year["_spanned_years"] = (
        one_year["_date_end"].dt.date - one_year["_date_start"].dt.date
    ).apply(lambda x: round(x.days / 365))
    warning_mask = (one_year["Commitment period (years)"] > 1) | (
        one_year["_spanned_years"] > 1
    )
    warnings = one_year[warning_mask]
    if not warnings.empty:
        print(
            "The following entities have 'One year commitment' with commitment "
            "period > 1 year\n"
            f"{warnings["Company"].drop_duplicates().to_list()}\n"
        )

    for currency, col in amount_columns.items():
        one_year[f"amount_{currency}"] = one_year[col]
    one_year["date_start"] = one_year["_date_start"]
    one_year["date_end"] = one_year["_date_end"]
    return one_year[get_final_cols(one_year)].copy()


def handle_annual_transfers(transfers: pd.DataFrame):
    annuals = transfers[
        transfers["Invoice preference"].str.strip().str.lower() == "annual"
    ].copy()
    annuals["_spanned_years"] = (
        annuals["_date_end"].dt.date - annuals["_date_start"].dt.date
    ).apply(lambda x: max(round(x.days / 365), 1))

    # Annual invoice transfers spanning more than 1 year should be split into
    # individual years... The raw data is supposed to be this way
    # ERRORS - Check the consistency of the commitment period and the support period
    error_mask = (annuals["_spanned_years"] > 1) & (
        annuals["_spanned_years"] != annuals["Commitment period (years)"]
    )
    errors = annuals[error_mask]
    if not errors.empty:
        print(
            "WARNING - The following Entities have Annual transfers spanning "
            "more than 1 year and inconsistent commitment period:\n"
            f"{errors["Company"].drop_duplicates().to_list()}\n"
        )

    to_split_mask = (annuals["_spanned_years"] > 1) & (
        annuals["_spanned_years"] == annuals["Commitment period (years)"]
    )
    to_split = annuals[to_split_mask].copy()

    to_split["_date_range"] = to_split.apply(
        lambda row: pd.date_range(
            row["_date_start"], row["_date_end"], freq="YS", inclusive="neither"
        ).append(pd.DatetimeIndex([row["_date_start"]]).sort_values()),
        axis=1,
    )
    splitted = to_split.explode("_date_range")
    splitted["_date_range_end"] = splitted["_date_range"].apply(
        lambda x: pd.to_datetime(f"{x.year}-12-31")
    )
    splitted["date_start"] = splitted[["_date_start", "_date_range"]].max(
        axis=1
    )
    splitted["date_end"] = splitted[["_date_end", "_date_range_end"]].min(
        axis=1
    )
    if not to_split.empty:
        print(f"Splitted {len(to_split)} annual transfers.")

    # Group back all Annual invoices
    defaults = annuals[~to_split_mask].copy()
    defaults["date_start"] = defaults["_date_start"]
    defaults["date_end"] = defaults["_date_end"]

    annuals_clean = pd.concat([splitted, defaults])
    for currency, col in amount_columns.items():
        annuals_clean[f"amount_{currency}"] = annuals_clean[col]

    return annuals_clean[get_final_cols(annuals_clean)].copy()


def handle_upfront_transfers(transfers: pd.DataFrame):
    # Re-group upfront transfers
    upfronts_base = transfers[
        transfers["Invoice preference"].str.strip().str.lower() == "upfront"
    ].copy()
    if upfronts_base.empty:
        return pd.DataFrame()

    compulsory_keys = [
        "Company",
        "Country",
        "Commitment period (years)",
        *amount_columns.values(),
    ]
    bonus_keys = ["Supporter type", "Agent", "Sponsorship level"]
    bonus_keys = [c for c in bonus_keys if c in transfers.columns]
    grouping_keys = compulsory_keys + bonus_keys
    grouped = upfronts_base.groupby(grouping_keys, dropna=False)
    upfronts = grouped.agg(
        date_start=pd.NamedAgg(column="_date_start", aggfunc="min"),
        date_end=pd.NamedAgg(column="_date_end", aggfunc="max"),
        number=pd.NamedAgg(column="_date_start", aggfunc="count"),
    )
    upfronts["original_ind"] = grouped.apply(
        lambda group: list(group.index), include_groups=False
    )
    upfronts.reset_index(inplace=True)
    upfronts["_spanned_years"] = (
        upfronts["date_end"].dt.date - upfronts["date_start"].dt.date
    ).apply(lambda x: max(round(x.days / 365), 1))

    # ERRORS - These are erroneous according to the discussed model
    mask_error = (
        upfronts["number"] != upfronts["Commitment period (years)"]
    ) | (upfronts["_spanned_years"] != upfronts["Commitment period (years)"])

    errors = upfronts[mask_error]
    if not errors.empty:
        print(
            "ERROR - There are some inconsistent data within the Upfront transfers\n"
            f"Check the following entities:\n{errors["Company"].drop_duplicates().to_list()}\n"
        )

    errors_ind = reduce(
        lambda a, b: a + b, errors["original_ind"].to_list(), []
    )
    upfronts_errors = upfronts_base[upfronts_base.index.isin(errors_ind)].copy()
    upfronts_errors["date_start"] = upfronts_errors["_date_start"]
    upfronts_errors["date_end"] = upfronts_errors["_date_end"]

    for currency, col in amount_columns.items():
        upfronts_errors[f"amount_{currency}"] = upfronts_errors[col]

    # For the remaining groups, the transfers are simply the calculated date start
    # & date end and the amount is annual_amount * commitment_years
    upfronts_correct = upfronts[~mask_error].copy()
    for currency, col in amount_columns.items():
        upfronts_correct[f"amount_{currency}"] = (
            upfronts_correct[col]
            * upfronts_correct["Commitment period (years)"]
        )

    upfronts_correct["Invoice preference"] = "Upfront"
    if not upfronts_correct.empty:
        print(f"Handled {len(upfronts_correct)} upfront transfers.")

    # Retrieve all data: transformed and untouched erroneous ones
    upfronts_clean = pd.concat(
        [
            upfronts_correct[get_final_cols(upfronts_correct)],
            upfronts_errors[get_final_cols(upfronts_errors)],
        ]
    )
    return upfronts_clean


def organize_transfers(df: pd.DataFrame):
    df["_date_start"] = pd.to_datetime(
        df[start_date_col], format="%d/%m/%Y", errors="raise"
    )
    df["_date_end"] = pd.to_datetime(
        df[end_date_col], format="%d/%m/%Y", errors="raise"
    )
    df["_spanned_time"] = df["_date_end"] - df["_date_start"]

    covered_statuses = [
        "annual",
        "one year commitment",
        "annual, one year commitment",
        "upfront",
    ]
    non_covered = df[
        ~df["Invoice preference"].str.strip().str.lower().isin(covered_statuses)
    ]
    if not non_covered.empty:
        raise Exception(
            f"Unhandled invoice preferences: {non_covered["Invoice preference"].drop_duplicates().to_list()}"
        )

    # Split data according to the Invoice preference
    res = pd.concat(
        [
            handle_one_year_transfers(df),
            handle_annual_transfers(df),
            handle_upfront_transfers(df),
        ]
    )

    # Map amount columns to amount & currency
    res["amount"] = None
    res["currency"] = None

    for c in amount_columns.keys():
        col = f"amount_{c}"
        mask = (~res[col].isna()) & (~res[col].isin([0, "0"]))
        res.loc[mask, "currency"] = c
        res.loc[mask, "amount"] = res[mask][col]
        res.drop(columns=[col], inplace=True)
    return res.sort_values(["Company", "date_start", "date_end"])


def discard_future_transfers(df: pd.DataFrame):
    current_year = date.today().year
    mask = df["date_start"] > pd.to_datetime(
        f"{current_year}-07-01", format="%Y-%m-%d"
    )
    futures = df[mask]
    if not futures.empty:
        print(f"Discarding {len(futures)} future transfers.")
    return df[~mask].copy()


def pre_process(export=False):
    df = pd.read_excel(f"{FILE_INPUT_FOLDER}/{INPUT_NAME}")

    # Find correct columns
    for regex in AMOUNT_COLS_REGEX:
        r = re.compile(regex)
        for c in df.columns:
            match = r.match(c)
            if match:
                amount_columns[match.group(1)] = c
    start_date_cols = [c for c in df.columns if "start date" in c]
    if len(start_date_cols) != 1:
        raise Exception(f"Ambiguous start date columns: {start_date_cols}")
    global start_date_col
    start_date_col = start_date_cols[0]
    end_date_cols = [c for c in df.columns if "end date" in c]
    if len(end_date_cols) != 1:
        raise Exception(f"Ambiguous end date columns: {end_date_cols}")
    global end_date_col
    end_date_col = end_date_cols[0]
    if "Invoice preferences" in df.columns:
        df.rename(
            columns={"Invoice preferences": "Invoice preference"}, inplace=True
        )

    df_filtered = select_transfers(df)
    df_clean = organize_transfers(df_filtered)
    df_final = discard_future_transfers(df_clean)
    df_final["date_start"] = df_final["date_start"].dt.date
    df_final["date_end"] = df_final["date_end"].dt.date
    if export:
        df_final.to_excel(
            f"{FILE_OUTPUT_FOLDER}/{OUTPUT_NAME}",
            sheet_name="Transfers",
            index=False,
        )
    return df_final


res = pre_process(export=False)

In [ ]:
# res.to_excel(
#     "/home/guillaume-alzieu/Nextcloud/TSOSI_data/doab/temp_correction.xlsx",
#     index=False,
# )
res.sort_values("amount", ascending=False)

### 2 - Pre-processed data -> ROR matching


In [ ]:
from pathlib import Path
import sys
import os
import django
import pandas as pd
from asgiref.sync import sync_to_async

# Add the parent directory to the system path and setup django
BASE_DIR = str(Path(os.getcwd()).resolve().parent.parent.parent.parent)

if BASE_DIR not in sys.path:
    sys.path.append(BASE_DIR)

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "backend_site.settings")

django.setup()

from tsosi.data.pid_matching import prepare_manual_matching


file_folder = Path.home() / "Nextcloud/TSOSI_data/doab/1_pre_processed/"
INPUT_NAME = "2025-02-11-DOAB_Sponsorship_Report_pre_processed.xlsx"
sheet = "Transfers"
name_column = "Company"
country_colum = "Country"


@sync_to_async
def to_run():
    xls = pd.ExcelFile(str(file_folder / INPUT_NAME))
    data = pd.read_excel(xls, sheet)
    return prepare_manual_matching(
        data, name_column, country_column=country_colum
    )


res = await to_run()

In [ ]:
export_folder = Path.home() / "Nextcloud/TSOSI_data/doab/2_matched/"
OUTPUT_NAME = "2025-02-11-DOAB_Sponsorship_Report_matched.xlsx"
res.to_excel(str(export_folder / OUTPUT_NAME), sheet_name=sheet, index=False)

In [ ]:
res[~res["Agent"].isna()][["Agent", "Country"]].drop_duplicates(
    ["Agent", "Country"]
).sort_values("Agent")

### 4 - Prepare enriched data


In [ ]:
from pathlib import Path
import sys
import os
import django
import pandas as pd

# Add the parent directory to the system path and setup django
BASE_DIR = str(Path(os.getcwd()).resolve().parent.parent.parent.parent)

if BASE_DIR not in sys.path:
    sys.path.append(BASE_DIR)

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "backend_site.settings")

django.setup()

from tsosi.data.utils import clean_null_values
from tsosi.data.preparation.cleaning_utils import clean_cell_value
from tsosi.data.pid_matching import process_enriched_data
from tsosi.models.transfer import (
    TRANSFER_ENTITY_TYPE_EMITTER,
    TRANSFER_ENTITY_TYPE_AGENT,
)

file_folder = Path.home() / "Nextcloud/TSOSI_data/doab/"
file_base_name = "2025-02-11-DOAB_Library_Report"
INPUT_NAME = f"3_enriched/{file_base_name}_enriched.xlsx"
sheet_transfers = "Transfers"
name_col = "Company"


def process(process_agents=False):
    xls = pd.ExcelFile(str(file_folder / INPUT_NAME))
    data = pd.read_excel(xls, sheet_transfers)
    # Process enriched emitter data
    res = process_enriched_data(data, name_col, TRANSFER_ENTITY_TYPE_EMITTER)

    if process_agents:
        # Process enriched agents data
        sheet_agent = "Consortiums"
        agent_col = "Agent"
        country_col = "Country"
        agents = pd.read_excel(xls, sheet_agent)
        for col in [agent_col, country_col]:
            res[col] = res[col].apply(clean_cell_value)
            agents[col] = agents[col].apply(clean_cell_value)
        # MAKE SURE THERE ARE NO DUPLICATES IN AGENT TO NOT CREATE TRANSFERS
        # WHEN LEFT JOINING
        agents.drop_duplicates(subset=[agent_col, country_col], inplace=True)
        clean_null_values(agents)
        clean_null_values(res)
        agents["_merged_from_agent"] = True
        res = res.merge(agents, on=[agent_col, country_col], how="left")
        # Check that all agents
        mask = ~res[agent_col].isna() & (res["_merged_from_agent"] != True)
        errors = res[mask]
        if not errors.empty:
            print(f"Error with agent handling, check return.")
            return errors
        res.drop(columns=["_merged_from_agent"], inplace=True)

        res = process_enriched_data(res, agent_col, TRANSFER_ENTITY_TYPE_AGENT)

    clean_null_values(res)
    res["date_start"] = res["date_start"].dt.date
    res["date_end"] = res["date_end"].dt.date

    return res


res = process(True)

In [ ]:
prepared_file_path = (
    file_folder / "4_prepared" / f"{file_base_name}_prepared.xlsx"
)

res.to_excel(str(prepared_file_path), sheet_name="Transfers", index=False)

### 5 - Generate TSOSI data file


In [ ]:
from pathlib import Path
import sys
import os
import django
from datetime import date

# Add the parent directory to the system path and setup django
BASE_DIR = str(Path(os.getcwd()).resolve().parent.parent.parent.parent)

if BASE_DIR not in sys.path:
    sys.path.append(BASE_DIR)

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "backend_site.settings")

django.setup()

from tsosi.data.preparation.doab.libraries import get_config as config_libraries
from tsosi.data.preparation.doab.sponsors import get_config as config_sponsors

date_data = date(2025, 2, 11)
# Libraries
file_path = (
    Path.home()
    / "Nextcloud/TSOSI_data/doab/4_prepared/2025-02-11-DOAB_Library_Report_prepared.xlsx"
)
sheet_name = "Transfers"
config = config_libraries(str(file_path), sheet_name, date_data)
config.generate_data_file()

# Sponsors
file_path = (
    Path.home()
    / "Nextcloud/TSOSI_data/doab/4_prepared/2025-02-11-DOAB_Sponsorship_Report_prepared.xlsx"
)
sheet_name = "Transfers"
config = config_sponsors(str(file_path), sheet_name, date_data)
config.generate_data_file()

### Use already performed matching


In [ ]:
## Map with previous matching
from pathlib import Path

file = (
    Path.home()
    / "Nextcloud/TSOSI_data/doab/2_matched"
    / "2025-02-11-DOAB_Library_Report_matched.xlsx"
)
matched = pd.read_excel(file)
matching_columns = [c for c in matched.columns if c.startswith("_")]
data = (
    matched.groupby(["Company", "Country"], dropna=False)[matching_columns]
    .first()
    .reset_index()
)

organized = res.merge(data, how="left", on=["Company", "Country"])
sorted_cols = []
for c in organized.columns:
    if c in matching_columns:
        continue
    sorted_cols.append(c)
    if c == "Company":
        sorted_cols += matching_columns

organized[sorted_cols]
organized["_processed"] = organized["_processed"].apply(
    lambda x: x if pd.isna(x) else True
)
organized[sorted_cols].to_excel(
    Path.home()
    / "Nextcloud/TSOSI_data/doab/2_matched"
    / "2025-02-11-DOAB_Library_Report_matched_grouped.xlsx",
    index=False,
)

### Use already enriched data


In [ ]:
import pandas as pd
from pathlib import Path

to_enrich_file = Path.home() / "Nextcloud/TSOSI_data/doab/temp_correction.xlsx"
enriched_file = (
    Path.home()
    / "Nextcloud/TSOSI_data/doab/3_enriched/2025-02-11-DOAB_Library_Report_enriched_backup.xlsx"
)

to_enrich = pd.read_excel(str(to_enrich_file))
enriched = pd.read_excel(str(enriched_file), sheet_name="Transfers")
consortiums = pd.read_excel(str(enriched_file), sheet_name="Consortiums")

enrichment_columns = [c for c in enriched.columns if c.startswith("_")]
data = (
    enriched.groupby(["Company", "Country"], dropna=False)[enrichment_columns]
    .first()
    .reset_index()
)

organized = to_enrich.merge(data, how="left", on=["Company", "Country"])
sorted_cols = []
for c in organized.columns:
    if c in enrichment_columns:
        continue
    sorted_cols.append(c)
    if c == "Company":
        sorted_cols += enrichment_columns

organized = organized[sorted_cols].copy()

In [ ]:
output_file = (
    Path.home()
    / "Nextcloud/TSOSI_data/doab/3_enriched/2025-02-11-DOAB_Library_Report_enriched.xlsx"
)
writer = pd.ExcelWriter(str(output_file), engine="xlsxwriter")
organized.to_excel(writer, sheet_name="Transfers", index=False)
consortiums.to_excel(writer, sheet_name="Consortiums", index=False)
writer.close()

In [ ]:
consortiums

### Bonus check


In [ ]:
# Check for overlapping periods
columns = [
    "Company",
    "_date_start",
    "_date_end",
    "Country",
    "Agent",
]
d = res[columns].copy(deep=True)

d["_date_range"] = d.apply(
    lambda row: pd.date_range(row["_date_start"], row["_date_end"]),
    axis=1,
)
d = d.explode("_date_range")
d[d[["Company", "_date_range"]].duplicated()]["Company"].unique()